# Polymerase Chain Assembly protocol for OT-2

In [ ]:
import opentrons.execute
from opentrons import protocol_api
import json

## General deck setup

In [ ]:
protocol = opentrons.execute.get_protocol_api("2.17")
protocol.home()

In [ ]:
# Load tip racks
tip_rack_20 = protocol.load_labware('opentrons_96_tiprack_20ul', 9)
tip_rack_300 = protocol.load_labware('opentrons_96_tiprack_300ul', 6)

In [1]:
# Load pipette tips
p20 = protocol.load_instrument(instrument_name='p20_single_gen2', mount="right", tip_racks = [tip_rack_20])
p300 = protocol.load_instrument(instrument_name='p300_single', mount='left', tip_racks=[tip_rack_300])
p300.flow_rate.aspirate = 50

In [ ]:
# Load well plates
frag_plate = protocol.load_labware("corning_96_wellplate_360ul_flat", 1)
qc_plate = protocol.load_labware("corning_96_wellplate_360ul_flat", 2)

### Add reagents to well plate

In [ ]:
# Define liquids (optional)
suspended_dna = protocol.define_liquid(
    name="Suspended DNA",
    description="Oligo fragments suspended in buffer",
    display_color="#a8329d",
)

In [ ]:
frag_wells = frag_plate.wells()[0:19]  # check that layout matches frag plate; currently goes A1, B1, C1,...

for well in frag_wells:
    well.load_liquid(suspended_dna, volume=50)

## Load modules

In [ ]:
thermocycler = protocol.load_module("thermocyclerModuleV1", 7)
thermocycler_plate = thermocycler.load_labware("nest_96_wellplate_100ul_pcr_full_skirt")

In [ ]:
temp_module = protocol.load_module("temperatureModuleV1", 3)
temp_tubes = temp_module.load_labware("opentrons_24_aluminumblock_generic_2ml_screwcap")

### Adding reagents to temperature controller

In [ ]:
# Define liquids (optional)
nuclease_free_water = protocol.define_liquid(
    name="Nuclease free water",
    description="Nuclease free water",
    display_color="#0000FF",
)

pcr_mastermix = protocol.define_liquid(
    name="PCR mastermix",
    description="PCR mastermix",
    display_color="#00FF00",
)

In [ ]:
# Define reagents
water_well = temp_tubes.wells_by_name()["A1"]
pcr_mastermix_well = temp_tubes.wells_by_name()["A2"]
waste_well = temp_tubes.wells_by_name()["A3"]

# Load liquids into the corresponding wells
water_well.load_liquid(liquid=nuclease_free_water, volume=200)
pcr_mastermix_well.load_liquid(liquid=pcr_mastermix, volume=200)

# Run protocol

### Assembly Operations Mapping

The `assembly_operations` dictionary provides a mapping of the different assembly stages and the corresponding well combinations for each stage. This mapping allows us to automate the pipetting operations required for the polymerase chain assembly (PCA) protocol.

The dictionary is structured as follows:
- Keys at the top level represent the assembly stages ("1", "2", "3", "4", "5").
- Within each stage, keys represent the destination well (e.g., "A1", "B1", "C1").
- Values are lists containing the wells or fragments to be combined in the destination well.

For example:
- In Assembly Stage 1, Well A1 contains fragments 1 and 2, Well A2 contains fragments 3 and 4, and so on.
- In Assembly Stage 2, Well B1 contains products from wells A1 and A2, Well B2 contains products from wells A3 and A4, and so on.

This pattern continues until the final assembled product is achieved in Well E1 during Operation 5.

In [3]:
with open('pca-assembly-operations.json', 'r') as file:
    assembly_operations = json.load(file)

In [ ]:
temp_module.set_temperature(celsius=4)

In [ ]:
thermocycler.open_lid()

 ### Assembly stage 1
 
First, add 50 uL of PCR mastermix to each of the initial wells in the thermocycler well-plate. See `pca-assembly-operations.json` for the layout of the thermocycler well plate. After each assembly operation, we'll ad 25 uL of PCR mastermix to the new reaction wells.

In [ ]:
def add_pcr_master_mix(protocol, assembly_step):
    step_data = assembly_operations[assembly_step]
    
    pcr_mastermix_vol = 50 if assembly_step == "1" else 25
    p300.pick_up_tip()
    
    for dest_well, source_wells in step_data["well_mapping"].items():
        if dest_well == source_wells[0]:
            # If destination well is the same as the first source well, skip the transfer
            continue
            
        dest_well_obj = thermocycler_plate.wells_by_name()[dest_well]
        
        p300.aspirate(pcr_mastermix_vol, pcr_mastermix_well.bottom(3))
        p300.dispense(pcr_mastermix_vol, dest_well_obj.top(z=2))
        p300.blow_out(dest_well_obj.top(z=5))
        
    p300.drop_tip()  # add all pcr mastermix with one tip

In [ ]:
add_pcr_master_mix(protocol, "1")

Now, we add the 25 uL of the suspended fragments to the appropriate wells on the thermocycler well plate. That means for the first assembly operation we have a total well volume of 100 uL (2 * 25 uL suspended fragments and 50 uL pcr mastermix). We also mix the liquid by aspirating/dispensing in the well, and blow out afterwards to ensure no droplets stay stuck on the tip.

In [ ]:
frag_vol_op1 = 25

def transfer_fragments_to_pcr_plate(protocol):
    # Comment the following lines to use a new tip for each transfer
    # p300.pick_up_tip()
    for dest_well, source_wells in assembly_operations["1"]["well_mapping"].items():
        for source_well in source_wells:
            frag_well = frag_plate.wells_by_name()[source_well]
            pcr_well = thermocycler_plate.wells_by_name()[dest_well]

            # Uncomment the following lines to use a new tip for each transfer
            p300.pick_up_tip()
            
            p300.aspirate(frag_vol_op1, frag_well.bottom(-3.5))
            p300.dispense(frag_vol_op1, pcr_well.top(z=2))
            p300.mix(2, 10, pcr_well.top(z=2))  # mix two times, 10 uL
            p300.blow_out(pcr_well.top(z=5))

            # Uncomment the following lines to use a new tip for each transfer
            p300.drop_tip()
            
    # Comment the following lines to use a new tip for each transfer
    # p300.drop_tip()

In [ ]:
transfer_fragments_to_pcr_plate(protocol)

Next, we run the first thermocycler profile. The initial denaturation and final extension are shared across all assembly operations, as is the number of cycles for the denaturation-annealing-extension (DAE) step. The part unique to each assembly operation is the extension time; we include the unique DAE profile in `pca-assembly-operations.json`.

After the thermocycling steps are complete, we turn off the lid heat and hold the block temperature at 4C while performing the next transfer operations.

In [ ]:
def run_thermocycler(protocol, assembly_step):
    step_data = assembly_operations[assembly_step]
    thermocycler.close_lid()
    
    # Set lid temperature to avoid condensation
    # https://physics.stackexchange.com/questions/488658/how-does-a-heated-lid-in-a-thermal-cycler-prevent-evaporation
    thermocycler.set_lid_temperature(105)
    
    init_denaturation_step = {"temperature": 98, "hold_time_seconds": 30}    
    final_extension_step = {"temperature": 72, "hold_time_seconds": 300}
    
    # Execute the thermocycler profile
    thermocycler.set_block_temperature(init_denaturation_step["temperature"], hold_time_seconds=init_denaturation_step["hold_time_seconds"])
    
    thermocycler.execute_profile(
        steps=step_data["thermocycling"]["dae_profile"],
        repetitions=30,
        block_max_volume=step_data["thermocycling"]["max_well_volume"]
    )
    
    thermocycler.set_block_temperature(final_extension_step["temperature"], hold_time_seconds=final_extension_step["hold_time_seconds"])
    
    thermocycler.deactivate_lid()
    
    if assembly_step == "1":
        thermocycler.set_block_temperature(temperature=4)
        
    thermocycler.open_lid()

In [ ]:
# run the first assembly operation
run_thermocycler(protocol, "1")

### Assembly stage 2
After the first assembly operation, add 25 uL PCR mastermix to the new reaction wells, then combine 12.5 uL of the assembly products in the new reaction wells, and then run another PCR cycler. That means for assembly stage 2-5 we'll have a total well volume of 50 uL (2*12.5 uL products of first assembly + 25 uL pcr mastermix).

In [ ]:
add_pcr_master_mix(protocol, "2")

In [ ]:
frag_vol_op2thru5 = 12.5

def transfer_assembly_products(protocol, assembly_step):
    step_data = assembly_operations[assembly_step]

    for dest_well, source_wells in step_data["well_mapping"].items():
        if dest_well == source_wells[0]:
            # If destination well is the same as the first source well, skip the transfer
            continue

        dest_well_obj = thermocycler_plate.wells_by_name()[dest_well]

        # Mix the contents of the source wells and then transfer to destination well
        for source_well in source_wells:
            source_well_obj = thermocycler_plate.wells_by_name()[source_well]
            p20.pick_up_tip()
            p20.mix(2, 10, source_well_obj.bottom(11))
            p20.aspirate(frag_vol_op2thru5, source_well_obj.bottom(11))
            p20.dispense(frag_vol_op2thru5, dest_well_obj.top(z=15))
            p20.mix(1, 10, dest_well_obj.bottom(11))
            p20.blow_out(dest_well_obj.bottom(18))
            p20.drop_tip()

In [ ]:
transfer_assembly_products(protocol, "2")

In [ ]:
run_thermocycler(protocol, "2")

### Assembly stage 3

In [ ]:
add_pcr_master_mix(protocol, "3")

In [ ]:
transfer_assembly_products(protocol, "3")

In [ ]:
run_thermocycler(protocol, "3")

### Assembly stage 4

In [ ]:
add_pcr_master_mix(protocol, "4")

In [ ]:
transfer_assembly_products(protocol, "4")

In [ ]:
run_thermocycler(protocol, "4")

### Assembly stage 5

In [ ]:
add_pcr_master_mix(protocol, "5")

In [ ]:
transfer_assembly_products(protocol, "5")

In [ ]:
run_thermocycler(protocol, "5")

# Utility commands

In [ ]:
def print_deck_layout(protocol):
    for slot, item in protocol.deck.items():
        if item:
            print(f"Slot {slot}: {item}")
        else:
            print(f"Slot {slot}: Empty")
            
print_deck_layout(protocol)

In [ ]:
p300.return_tip()